In [4]:
import spacy
import datetime as dt
from dateutil.relativedelta import relativedelta
from datetime import date
import dateutil.parser as parser
from datetime import timedelta
from flair.models import SequenceTagger
from flair.data import Sentence
from dateparser import parse
import re

In [482]:
# possible times:
# 20 days ago
# 20 days from now
# yesterday
# January 1st 
# twenty days ago
# a week ago 
# a month ago
# today
# early Saturday morning
# last evening 
# last Saturday 
# last Saturday evening 
# this Saturday vs last Saturday 
# about Saturday noon
# around noon Saturday
# around lunchtime
# about noon
# October last year
# the end of June
# the beginning of June
# earlier this month
# late last month
# around 3 am
# between 3 am and 5 am
# 6:50 a.m . Monday
# # about 6pm last night
# this past weekend
# last weekend
# weekend
# beginning of this week
# late last week
# last week last month 
# last week this month 
# last weekend of last month
# this winter / last winter
# several/some/a couple years ago 

# MIGHT add 
# Less than 24 hours
# about an hour ago 

In [7]:
help_dict = {
    'one': 1,
    'two': 2,
    'three': 3,
    'four': 4,
    'five': 5,
    'six': 6,
    'seven': 7,
    'eight': 8,
    'nine': 9,
    'zero' : 0,
    'a': 1,
    'ten': 10,
    'eleven': 11,
    'twelve': 12,
    'thirteen': 13,
    'fourteen': 14,
    'fifteen': 15,
    'sixteen': 16,
    'seventeen': 17,
    'eighteen': 18, 
    'nineteen': 19, 
    'twenty': 20, 
    'thirty': 30,
    'forty': 40,
    'fifty': 50,
    'sixty': 60,
    'seventy': 70,
    'eighty': 80,
    'ninety':90, 
    '0': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    '7': 7,
    '8': 8,
    '9': 9,
    '10': 10,
    '11': 11,
    '12': 12,
    '13': 13,
    '14': 14,
    '15': 15,
    '16': 16,
    '17': 17,
    '18': 18,
    '19': 19,
    '20': 20
}

week_days = {'Monday': 1,
                'Tuesday': 2,
                'Wednesday': 3,
                'Thursday': 4,
                'Friday': 5,
                'Saturday': 6,
                'Sunday': 7}
weekdays_reversed = {v: k for k, v in week_days.items()} 
times_of_day = {'morning':(5, 11),
                'afternoon': (12, 17),
                'evening': (18, 22),
                'night': (23, 4),
                'lunchtime': (12, 15), 
                'noon': (12, 17),
                'midnight': (24, 2)
                 }

months = {'January': 1,
         'February': 2,
         'March': 3,
         'April': 4,
         'May': 5,
         'June': 6,
         'July': 7,
         'August': 8,
         'September': 9,
         'October': 10,
         'Novemver': 11,
         'December': 12}
months_reversed = {v: k for k, v in months.items()}
seasons = {'winter': (12, 2),
           'spring': (3, 5),
           'summer': (6, 8),
           'autumn': (9, 11),
           'fall': (9, 11)}


In [483]:
date_example_1 = '16 Jan 2020, 5:45am'
date_example_2 = 'JANUARY 13, 2019'
date_example_3 = 'Dec 1, 2021, 7:12AM PST'
date_example_4 = 'May 23, 2022, 7:12AM PST'

# If today is Friday and we need to find the number of day between this Friday and last Monday, we pass the current date
# and the number of day we want to find the difference with(Monday - 1)

def parse_date_weekday(date, day_number, modifier):
    '''
    given a number of the day of the week returns the difference between the date and this particular day
    
    '''
    parsed_date = parser.parse(date)
    year = parsed_date.year
    month = parsed_date.month
    day = parsed_date.day
    weekday = parsed_date.weekday() + 1
#     print(weekday)
    if modifier == 'last':
        delta = 7 + weekday - day_number
    else:
        delta = weekday - day_number 
        if delta < 0:
            return None
    return delta
    
print(parse_date_weekday(date_example_4, 1, 'last'))
print(parse_date_weekday('2022-06-07', 5, 'last'))
print(parse_date_weekday('2022-06-07', 1, ''))
print(parse_date_weekday('2022-06-07', 1, 'last'))

7
4
1
8


In [280]:
# seasons
def seasons_interval(phrase, date_string):
    
    winter = False
    spring = False
    summer = False
    autumn = False
    last_season = False
    
    if 'winter' in phrase:
        winter = True
    if 'autumn' in phrase or 'fall' in phrase:
        autumn = True
    if 'summer' in phrase:
        summer = True 
    if 'spring' in phrase:
        spring = True 
    if 'last' in phrase or 'past' in phrase:
        last_season = True
        
    if last_season:
        if winter:
            date_1, date_2 = month_parsing('last ' + 'December', date_string) 
            date_2 = date_1 + relativedelta(months=3)
        if spring:
            date_1, date_2 = month_parsing('last ' + 'March', date_string)
            date_2 = date_1 + relativedelta(months=3)
        if summer:
            date_1, date_2 = month_parsing('last ' + 'June', date_string)
            date_2 = date_1 + relativedelta(months=3)
        if autumn:
            date_1, date_2 = month_parsing('last ' + 'September', date_string)
            date_2 = date_1 + relativedelta(months=3)
    else:
        if winter:
            date_1, date_2 = month_parsing('this ' + 'December', date_string) 
            date_2 = date_1 + relativedelta(months=3)
        if spring:
            date_1, date_2 = month_parsing('this ' + 'March', date_string)
            date_2 = date_1 + relativedelta(months=3)
        if summer:
            date_1, date_2 = month_parsing('this ' + 'June', date_string)
            date_2 = date_1 + relativedelta(months=3)
        if autumn:
            date_1, date_2 = month_parsing('this ' + 'September', date_string)
            date_2 = date_1 + relativedelta(months=3)
            
    return date_1, date_2
        
            
            
print(seasons_interval("this spring", '2022-06-06'))        
print(seasons_interval("last spring", '2022-06-06'))
print(seasons_interval("last summer", '2022-06-06'))  
print(seasons_interval("this summer", '2022-06-06')) 
print(seasons_interval("last winter", '2022-06-06'))
print(seasons_interval("summer", '2022-06-06'))
print(seasons_interval("this winter", '2022-06-06'))
    

(datetime.date(2022, 3, 1), datetime.date(2022, 6, 1))
(datetime.date(2022, 3, 1), datetime.date(2022, 6, 1))
(datetime.date(2021, 6, 1), datetime.date(2021, 9, 1))
(datetime.date(2022, 6, 1), datetime.date(2022, 9, 1))
(datetime.date(2021, 12, 1), datetime.date(2022, 3, 1))
(datetime.date(2022, 6, 1), datetime.date(2022, 9, 1))
(datetime.date(2021, 12, 1), datetime.date(2022, 3, 1))


In [69]:
regex = r"[0-9]+:?[0-9]* (am|a.m.|a.m .|AM|pm|p.m.|PM)"
phrase = "06:50 a.m ."
lst = []
for result in re.finditer(regex, phrase):
    lst.append(result.group())
print(lst)

['06:50 a.m ']


In [484]:
# date = '2022-06-01'
def week(phrase, date_string):
    beginning = False
    middle = False
    end = False
    last = False
    date_1 = None
    split_phrase = phrase.split(' ')
    parsed_date = parser.parse(date_string)
    for i, word in enumerate(split_phrase):
        if word == 'week' and i <= len(split_phrase) - 1 and (split_phrase[i-1] == 'last' or split_phrase[i-1] == 'past'):
            last = True 
    if 'beginning' in phrase or 'early' in phrase:
        beginning = True
    if 'middle' in phrase:
        middle = True
    if 'end ' in phrase or 'late' in phrase:
        end = True 
    if last:
        date_1, _, _ = last_and_this_day_calculation("last Monday", date_string)
    else:
        date_1, _, _ = last_and_this_day_calculation("Monday", date_string)
        
    date_start = parsed_date - relativedelta(days=date_1)
    date_end = date_start + relativedelta(days=6) + relativedelta(minutes=1439)
    
    if beginning:
        date_end = date_start + relativedelta(days=2)
    if end:
        date_start = date_start + relativedelta(days=3)
    if middle:
        date_start = date_start + relativedelta(days=2)
        date_end = date_end - relativedelta(days=2)
    return date_start, date_end


print(week("last week", '2022-06-07')) 
print(week("late last week", '2022-06-06')) 
print(week("this week", '2022-06-07'))
print(week("late last week", '2022-06-06')) 
print(week("in the beginning of this week", '2022-06-08'))
print(week("middle of last week", '2022-06-06'))
print(week("end of last week", '2022-06-06'))

(datetime.datetime(2022, 5, 30, 0, 0), datetime.datetime(2022, 6, 5, 23, 59))
(datetime.datetime(2022, 6, 2, 0, 0), datetime.datetime(2022, 6, 5, 23, 59))
(datetime.datetime(2022, 6, 6, 0, 0), datetime.datetime(2022, 6, 12, 23, 59))
(datetime.datetime(2022, 6, 2, 0, 0), datetime.datetime(2022, 6, 5, 23, 59))
(datetime.datetime(2022, 6, 6, 0, 0), datetime.datetime(2022, 6, 8, 0, 0))
(datetime.datetime(2022, 6, 1, 0, 0), datetime.datetime(2022, 6, 3, 23, 59))
(datetime.datetime(2022, 6, 2, 0, 0), datetime.datetime(2022, 6, 5, 23, 59))


In [228]:
date = '2022-06-03'
# does not predict anything if this weekend has not started, thus if today is not Sunday
def weekend(phrase, date_string):
    beginning = False
    middle = False
    end = False
    last = False
    date_1 = None
    date_2 = None
    
    if 'beginning' in phrase or 'early' in phrase:
        beginning = True
    if 'middle' in phrase:
        middle = True
    if ' end' in phrase or 'late' in phrase:
        end = True
    if 'last' in phrase or 'past' in phrase:
        last = True   
    if 'weekend' in phrase:
        if last:
            date_1, _, _ = last_and_this_day_calculation("last Saturday", date_string)
            date_2, _, _ = last_and_this_day_calculation("last Sunday", date_string)
        else: 
            date_1, _, _ = last_and_this_day_calculation("Saturday", date_string)
            date_2, _, _ = last_and_this_day_calculation("Sunday", date_string)
        parsed_date = parser.parse(date_string)
        date_1 = parsed_date - relativedelta(days=date_1)
        date_2 = parsed_date - relativedelta(days=date_2) + relativedelta(minutes=1439)
        if beginning:
            date_2 = date_1 + relativedelta(hours=12)
        if end:
            date_1 = date_1 + relativedelta(hours=36)
        if middle:
            date_1 = date_1 + relativedelta(hours=15)
            date_2 = date_2 - relativedelta(hours=15)
        return date_1, date_2
         
print(weekend('last weekend', '2022-06-06'))
print(weekend('weekend', '2022-06-06'))
print(weekend('this past weekend', '2022-06-06'))
print(weekend('early past weekend', '2022-06-06'))
print(weekend('in the beginning of the last weekend', '2022-06-06'))
print(weekend('in the end of the last weekend', '2022-06-06'))
print(weekend('late last weekend', '2022-06-06'))


(datetime.datetime(2022, 6, 4, 0, 0), datetime.datetime(2022, 6, 5, 23, 59))
(datetime.datetime(2022, 6, 4, 0, 0), datetime.datetime(2022, 6, 5, 23, 59))
(datetime.datetime(2022, 6, 4, 0, 0), datetime.datetime(2022, 6, 5, 23, 59))
(datetime.datetime(2022, 6, 4, 0, 0), datetime.datetime(2022, 6, 4, 12, 0))
(datetime.datetime(2022, 6, 4, 0, 0), datetime.datetime(2022, 6, 4, 12, 0))
(datetime.datetime(2022, 6, 5, 12, 0), datetime.datetime(2022, 6, 5, 23, 59))
(datetime.datetime(2022, 6, 5, 12, 0), datetime.datetime(2022, 6, 5, 23, 59))


In [485]:
regex_time = r"[0-9]+:?[0-9]* (am|a.m.|a.m .|AM|pm|p.m.|PM)"
def time(phrase):
    around = False
    about = False
    between = False
    lst_time = []
    time_1 = None 
    time_2 = None
    if 'around' in phrase:
        around = True 
        phrase = phrase.replace('around', '')
    if 'about' in phrase:
        about = True 
        phrase = phrase.replace('about', '')
    if 'between' in phrase:
        between = True 
    for result in re.finditer(regex_time, phrase):
        lst_time.append(result.group())
        
    if len(lst_time) == 1:
        time_1 = parse(lst_time[0])
    if len(lst_time) == 2:
        time_1 = parse(lst_time[0])
        time_2 = parse(lst_time[1])
   
    if around or about or len(lst_time) == 1:
        time_1_1 = time_1 - relativedelta(minutes=30)
        time_1_2 = time_1 + relativedelta(minutes=30)
        return time_1_1, time_1_2
    elif between and len(lst_time) == 2:
        return time_1, time_2
    
              
print(time('between 3 am and 5 am')) 
print(time('between 3 am and 5 pm')) 
print(time('around 3 pm'))
print(time('about 3 am'))
print(time('6:50 a.m . Monday'))
print(time('last night'))

(datetime.datetime(2022, 6, 9, 3, 0), datetime.datetime(2022, 6, 9, 5, 0))
(datetime.datetime(2022, 6, 9, 3, 0), datetime.datetime(2022, 6, 9, 17, 0))
(datetime.datetime(2022, 6, 9, 14, 30), datetime.datetime(2022, 6, 9, 15, 30))
(datetime.datetime(2022, 6, 9, 2, 30), datetime.datetime(2022, 6, 9, 3, 30))
(datetime.datetime(2022, 6, 9, 6, 20), datetime.datetime(2022, 6, 9, 7, 20))
None


In [486]:
# last March == in March, but it is not the same as in March last year 

d = '2022-06-03'
def month_parsing(phrase, date_string):
    if 'month' in phrase:
        month = month_transformation(phrase)
        phrase = phrase.replace('month', month)
    phrase_list = phrase.split(' ')

    month = None
    anchor = ''
    last_year = False
    for i, word in enumerate(phrase_list):
        if word in months.keys():
            month = months[word]
            if i - 1 >= 0 and phrase_list[i-1] == 'last':
                anchor = 'last'
        if word == 'year' and i > 0  and phrase_list[i-1] == 'last':
            last_year = True
            
    parsed_date = parser.parse(date_string) 
    
    if month > parsed_date.month:
        anchor = 'last'
    
    
    if last_year == True or (month >= parsed_date.month and anchor):
        year = parsed_date.year-1

    else:
        year = parsed_date.year
 
    new_date_1 = dt.date(year, month, 1)
    new_date_2 = new_date_1 + relativedelta(months=1) - relativedelta(days=1)
    
    if 'beginning' in phrase:
        new_date_2 = new_date_1 + relativedelta(days=10)
    elif 'middle' in phrase:
        new_date_1 = dt.date(year, month, 10)
        new_date_2 = new_date_1 + relativedelta(days=10)
    elif 'end' in phrase:
        new_date_1 = dt.date(year, month, 20)
        new_date_2 = new_date_1 + relativedelta(days=10)
    elif 'late' in phrase or 'later' in phrase:
        new_date_1 = dt.date(year, month, 15)
        new_date_2 = new_date_1 + relativedelta(days=15)
    elif 'early' in phrase or 'earlier' in phrase: 
        new_date_2 = new_date_1 + relativedelta(days=15)

    return new_date_1, new_date_2 

 
print(month_parsing('last October', '2022-06-03'))
print(month_parsing('October last year', '2022-06-03'))
print(month_parsing('in October', '2022-06-03'))
print(month_parsing('the end of June', '2022-06-03'))
print(month_parsing('the middle of June', '2022-06-03')) 
print(month_parsing('last June', '2022-06-03'))
print(month_parsing('the middle of June last year', '2022-06-03')) 
print(month_parsing('this month', '2022-06-03')) 
print(month_parsing('last month', '2022-06-03'))
print(month_parsing('early this month', '2022-06-03')) 
print(month_parsing('late this month', '2022-06-03'))
print(month_parsing('the end of May', '2022-06-03'))
print(month_parsing('last April', '2022-06-03'))
print(month_parsing('last June', '2022-06-03'))
print(month_parsing('this June', '2022-06-03'))


(datetime.date(2021, 10, 1), datetime.date(2021, 10, 31))
(datetime.date(2021, 10, 1), datetime.date(2021, 10, 31))
(datetime.date(2021, 10, 1), datetime.date(2021, 10, 31))
(datetime.date(2022, 6, 20), datetime.date(2022, 6, 30))
(datetime.date(2022, 6, 10), datetime.date(2022, 6, 20))
(datetime.date(2021, 6, 1), datetime.date(2021, 6, 30))
(datetime.date(2021, 6, 10), datetime.date(2021, 6, 20))
(datetime.date(2022, 6, 1), datetime.date(2022, 6, 30))
(datetime.date(2022, 5, 1), datetime.date(2022, 5, 31))
(datetime.date(2022, 6, 1), datetime.date(2022, 6, 16))
(datetime.date(2022, 6, 15), datetime.date(2022, 6, 30))
(datetime.date(2022, 5, 20), datetime.date(2022, 5, 30))
(datetime.date(2022, 4, 1), datetime.date(2022, 4, 30))
(datetime.date(2021, 6, 1), datetime.date(2021, 6, 30))
(datetime.date(2022, 6, 1), datetime.date(2022, 6, 30))


In [488]:

def month_transformation(month_phrase, date_string):
    
    '''
    given the date, identifies what month is meant in the str phrase
    '''
    phrase_list = month_phrase.split(' ')
    month = None
    anchor = ''
    month_index = len(phrase_list) - 1
    for i, word in enumerate(phrase_list):
        if word == 'month':
            month_index = i
        if i >= 1 and phrase_list[i-1] == 'last' and word == 'month':
            anchor = 'last'
    parsed_date = parser.parse(date_string)
    if anchor:
        month = parsed_date.month - 1
        return months_reversed[month]
    else:
        month = parsed_date.month
        return months_reversed[month]
    
    
    
print(month_transformation('last month', '2022-06-03'))
print(month_transformation('this month', '2022-06-03'))


May
June


In [82]:
def date_and_time_identifier(phrase):
    phrase_list = phrase.split(' ')
    date = None
    date_phrase = None
    time = None
    time_phrase = None
    for word in phrase_list:
        if word in times_of_day.keys():
            time = True 
            time_phrase = word
        if word in week_days.keys():
            date = True 
            date_phrase = word
    return time_phrase, date_phrase

print(date_and_time_identifier('last Saturday'))   
print(date_and_time_identifier('last Saturday evening'))  
print(date_and_time_identifier('about Saturday noon')) 
print(date_and_time_identifier('this morning')) 


(None, 'Saturday')
('evening', 'Saturday')
('noon', 'Saturday')
('morning', None)


In [489]:
# date = '2022-05-29'
def last_and_this_day_calculation(phrase, date_string):
    '''
    given a date, returns a tuple with a time delta between the str date and the one mentioned in the phrase,
    the hour for the start_date and the hour for the end_date
    
    '''
    
    time_phrase, date_phrase = date_and_time_identifier(phrase)
    days_delta, time_start, time_delta = None, None, None
    if date_phrase:
        day_number = week_days[date_phrase]
        current_weekday = parse(date_string).weekday() + 1
        if 'last' in phrase or 'previous' in phrase or day_number >= current_weekday:
            days_delta = parse_date_weekday(date_string, day_number, 'last')
        else:
            days_delta = parse_date_weekday(date_string, day_number, '')
    if 'next' in phrase:
        days_delta = None
    if time_phrase and not date_phrase:
        if 'last' in phrase:
            days_delta = 1
        if 'this' in phrase:
            days_delta = 0
    if time_phrase:
        time_start = times_of_day[time_phrase][0]
        time_end = times_of_day[time_phrase][1]
        if time_start > time_end: 
            time_delta = 24 - time_start + time_end
        else:
            time_delta = time_end - time_start 

        if 'early' in phrase:
            time_delta = time_delta / 2
        if 'late' in phrase: 
            time_delta = time_delta / 2
            time_start = time_start + time_delta

    return days_delta, time_start, time_delta

print(last_and_this_day_calculation('last Saturday', '2022-05-29'))      
print(last_and_this_day_calculation('last Saturday evening', '2022-05-29')) 
print(last_and_this_day_calculation('this Monday morning', '2022-05-29')) 
print(last_and_this_day_calculation('late Tuesday evening', '2022-05-29')) 
print(last_and_this_day_calculation('early Tuesday evening', '2022-05-29'))
print(last_and_this_day_calculation('about Wednesday noon', '2022-05-29'))
print(last_and_this_day_calculation('around noon Wednesday', '2022-05-29'))
print(last_and_this_day_calculation('this morning', '2022-05-29'))
print(last_and_this_day_calculation('last morning', '2022-05-29'))
print(last_and_this_day_calculation('6:50 a.m . Monday', '2022-05-29'))
print(last_and_this_day_calculation('last night', '2022-05-29'))
print(last_and_this_day_calculation('Saturday', '2022-06-07')) 
print(last_and_this_day_calculation('last Saturday', '2022-06-07')) 
print(last_and_this_day_calculation('next Saturday', '2022-05-29')) 
print(last_and_this_day_calculation('morning', '2022-05-29')) 
print(last_and_this_day_calculation('last night', '2022-05-29'))
# next Saturday should return None 
# on Saturday should return last saturday 

(8, None, None)
(8, 18, 4)
(6, 5, 6)
(5, 20.0, 2.0)
(5, 18, 2.0)
(4, 12, 5)
(4, 12, 5)
(0, 5, 6)
(1, 5, 6)
(6, None, None)
(1, 23, 5)
(3, None, None)
(3, None, None)
(None, None, None)
(None, 5, 6)
(1, 23, 5)


In [273]:
print(last_and_this_day_calculation('last Monday', '2022-06-07')) 
print(last_and_this_day_calculation('this Monday', '2022-06-07')) 

(8, None, None)
(1, None, None)


In [352]:
def days_time_triplet(phrase, date_string):
    days_delta, time_start, time_delta = last_and_this_day_calculation(phrase, date_string)
    event_date = None
    date_start, date_end = None, None
    publication_date = parse(date_string)
    if days_delta:
        event_date = parser.parse(date_string) - relativedelta(days=days_delta)
        date_start = event_date
        date_end = event_date + relativedelta(hours=23, minutes=59)
    if days_delta and time_start:
        date_start = event_date + relativedelta(hours=time_start)
        date_end = date_start + relativedelta(hours=time_delta)
    elif time_start:
        date_start = dt.date(publication_date.year, publication_date.month, publication_date.day) + relativedelta(hours=time_start)
        date_end = date_start + relativedelta(hours=time_delta)
        
    return date_start, date_end
    
print(days_time_triplet('Saturday', '2022-06-07'))
print(days_time_triplet('Saturday evening', '2022-06-07'))
print(days_time_triplet('late Saturday evening', '2022-06-07'))
print(days_time_triplet('in the morning', '2022-06-07'))
print(days_time_triplet('this Monday', '2022-06-08'))
print(days_time_triplet('last Monday', '2022-06-08'))
print(days_time_triplet('Monday', '2022-06-08'))
print(days_time_triplet('6 pm Monday', '2022-06-08'))
# print(days_time_triplet('6 pm', '2022-06-08'))
# print(days_time_triplet('2 days ago', '2022-06-08'))

    

(datetime.datetime(2022, 6, 4, 0, 0), datetime.datetime(2022, 6, 4, 23, 59))
(datetime.datetime(2022, 6, 4, 18, 0), datetime.datetime(2022, 6, 4, 22, 0))
(datetime.datetime(2022, 6, 4, 20, 0), datetime.datetime(2022, 6, 4, 22, 0))
(datetime.datetime(2022, 6, 7, 5, 0), datetime.datetime(2022, 6, 7, 11, 0))
(datetime.datetime(2022, 6, 6, 0, 0), datetime.datetime(2022, 6, 6, 23, 59))
(datetime.datetime(2022, 5, 30, 0, 0), datetime.datetime(2022, 5, 30, 23, 59))
(datetime.datetime(2022, 6, 6, 0, 0), datetime.datetime(2022, 6, 6, 23, 59))
(datetime.datetime(2022, 6, 6, 0, 0), datetime.datetime(2022, 6, 6, 23, 59))


In [490]:
import datetime
from datetime import datetime
def exact_date(phrase, date_string):
    
    'given the date in the past in a string, returns time delta between this string and parsed date'
    
    parsed_phrase = parse(phrase)
    phrase_month = parsed_phrase.month
    phrase_day = parsed_phrase.day
    phrase_year = parsed_phrase.year
    parsed_date = parser.parse(date_string)
    current_year = parse('now').year
    
    if current_year != phrase_year:
        phrase_date = dt.date(phrase_year, phrase_month, phrase_day)
    else:
        if parsed_date.month - phrase_month <= 0:
            if parsed_date.month - phrase_month == 0 and parsed_date.day - phrase_day > 0:
                phrase_date = dt.date(parsed_date.year, phrase_month, phrase_day)
            else:
                
                phrase_date = dt.date(parsed_date.year - 1, phrase_month, phrase_day)
        else:
            phrase_date = dt.date(parsed_date.year, phrase_month, phrase_day)

    parsed_date = dt.date(parsed_date.year, parsed_date.month, parsed_date.day)
    delta = parsed_date - phrase_date
    return delta.days

print(exact_date('1 June', '2022-06-06'))    
print(exact_date('15 Dec', '2022-06-06'))
print(exact_date('15 Dec', '2022-05-17'))
print(exact_date('7 June', '2022-06-06')) 
print(exact_date('Dec. 15 , 2020', '2022-06-06')) 
print(exact_date('in 2020', '2022-06-06')) 
print(exact_date('Dec. 15 , 2022', '2022-06-06')) 
print(exact_date('Dec. 15 , 2021', '2022-06-06'))
print(exact_date('Mar. 15 , 2022', '2022-06-06'))


5
173
153
364
538
727
173
173
83


In [491]:
date_string = '2022-06-06'

def days_count(phrase, date_string):
    
    def is_today():
        try:
            return parser.parse(phrase).date() == datetime.today().date()
        except:
            return False
    
    if is_today():
       return None
    
    if 'yesterday' in phrase:
        return 1
    elif 'today' in phrase or 'tonight' in phrase:
        return 0
    
    unit = None
    try:
        days = exact_date(phrase, date_string)
        return days
    except:
        pass

        
    tokenized_phrase = phrase.split(' ')    
    for i in range(len(tokenized_phrase)):
        if tokenized_phrase[i] in help_dict.keys():
            tokenized_phrase[i] = help_dict[tokenized_phrase[i]]
    if 'ago' in phrase or 'from now' in phrase or 'before' in phrase:

        if 'day' in tokenized_phrase or 'days' in tokenized_phrase:
            unit = 1
        elif 'week' in tokenized_phrase or 'weeks' in tokenized_phrase:
            unit  = 7
        elif 'month' in tokenized_phrase or 'months' in tokenized_phrase:
            unit = 30
        elif 'year' in tokenized_phrase or 'years' in tokenized_phrase:
            unit = 365
        for item in tokenized_phrase:
            if type(item) == int:
                return unit * item 
    
    else:
        return None
    

print(days_count('nine days ago', date_string))   
print(days_count('weekend', date_string))
print(days_count('20 days from now', date_string))
print(days_count('20 days ago', date_string))
print(days_count('a month ago', date_string))
print(days_count('a week ago', date_string))
print(days_count('1 May', date_string))
print(days_count('23 Jan', date_string))
print(days_count('1 Dec', date_string))
print(days_count('two years ago', date_string))
print(days_count('several years ago', date_string))
print(days_count('a year ago', date_string))
print(days_count('seven years ago', date_string))
print(days_count('this week', date_string))
print(days_count('2 pm', date_string))


6
None
20
17
28
4
36
134
187
727
None
362
2554
362
None


In [492]:
def specific_year(phrase):
    phrase_date = parse(phrase)
    date_start = dt.date(phrase_date.year, 1, 1)
    date_end = date_start + relativedelta(days=364)
    return date_start, date_end

print(specific_year('in 2012'))

(datetime.date(2012, 1, 1), datetime.date(2012, 12, 30))


In [302]:
# between July and August
# between 2005 and 2008
# between last evening and last midnight
# between the morning and the afternoon 
# between 3rd and 7th of August ---- ?
# in 2005

def between_phrase(phrase, date_string):
    
    month = False
    year = False
    part_of_day = False 
    weekday = False
    season = False
    
    between_regex = r"(?<=between )(.*? and .*?$)"
    date_phrases = []
    for result in re.finditer(between_regex, phrase):
        date_phrases.append(result.group())
    date_phrases = date_phrases[0].replace('and', ',').split(',')
    for date in date_phrases:
        try:
            year_local = int(date)
            year = True
        except:
            for month_local in months.keys():
                if month_local in date:
                    month = True
            for time_local in times_of_day.keys(): 
                if time_local in date:
                    part_of_day = True
            for day_local in week_days.keys():
                if day_local in date:
                    weekday = True
            for season_local in seasons.keys():
                if season_local in date:
                    season = True
                    
    date_start = None
    date_end = None
                    
    if month:
        date_start, _ = month_parsing(date_phrases[0], date_string)
        _, date_end = month_parsing(date_phrases[1], date_string)
    if part_of_day or weekday:
        date_start, _ = days_time_triplet(date_phrases[0], date_string)
        _, date_end = days_time_triplet(date_phrases[1], date_string)
    if season:
        date_start, _  = seasons_interval(date_phrases[0], date_string)
        _, date_end  = seasons_interval(date_phrases[1], date_string)
        pass
    if year:
        date_start, _ = specific_year(date_phrases[0])
        _, date_end = specific_year(date_phrases[1])
        
    return date_start, date_end
    
    
    
print(between_phrase('between July and August', date_string))
print(between_phrase('between last evening and last night', '2022-06-06'))
print(between_phrase('between winter and spring', '2022-06-06'))
print(between_phrase('between 2010 and 2012', '2022-06-06'))    

(datetime.date(2021, 7, 1), datetime.date(2021, 8, 31))
(datetime.datetime(2022, 6, 5, 18, 0), datetime.datetime(2022, 6, 6, 4, 0))
(datetime.date(2021, 12, 1), datetime.date(2022, 6, 1))
(datetime.date(2010, 1, 1), datetime.date(2012, 12, 30))


In [505]:
# several/a couple/some years ago
# several/a couple/some weeks ago
# several/a couple/some days ago
# several/a couple/some months ago
def years_interval(phrase, date_string):
    vague_phrase = False
    index = None
    phrase_list = phrase.split(' ')
    time_unit = None
    for i, word in enumerate(phrase_list):
        if word == 'years':
            time_unit = 'years'
            index = i
        elif word == 'months':
            time_unit = 'months'
            index = i
        elif word == 'weeks':
            time_unit = 'weeks'
            index = i
        elif word == 'days':
            time_unit = 'days'
            index = i
    phrase_before = ' '.join(phrase_list[0:i+1])
    if 'several' in phrase_before or 'couple' in phrase_before or 'some' in phrase_before:
        vague_phrase = True 
    if vague_phrase:
        date_start, _ = time_interval_with_number_of_days(f'six {time_unit} ago', date_string)
        _, date_end = time_interval_with_number_of_days(f'three {time_unit} ago', date_string)
    return date_start, date_end
    
print(years_interval('several years ago', '2022-06-04')) 
print(years_interval('a couple of days ago', '2022-06-04'))
print(years_interval('some months ago', '2022-06-04'))

(datetime.datetime(2016, 6, 9, 0, 0), datetime.datetime(2019, 6, 9, 23, 59))
(datetime.datetime(2022, 6, 3, 0, 0), datetime.datetime(2021, 6, 6, 23, 59))
(datetime.datetime(2021, 12, 9, 0, 0), datetime.datetime(2022, 3, 9, 23, 59))


In [434]:
def time_interval_with_number_of_days(phrase, date_string):
    try:
        number_of_days_to_subtract = days_count(phrase, date_string)
#print(number_of_days_to_subtract)
#     confidence_interval = None
#     if number_of_days_to_subtract <= 30:
#         confidence_interval = 0
#     elif number_of_days_to_subtract > 30 and number_of_days_to_subtract <= 365:
#         confidence_interval = 1
#     else:
#         confidence_interval = 2
        
        date_start = parser.parse(date_string) - relativedelta(days=number_of_days_to_subtract)
        date_end = date_start + relativedelta(hours=23, minutes=59)
        return date_start, date_end
    except:
        return None, None

print(time_interval_with_number_of_days('two days ago', '2022-06-04'))
print(time_interval_with_number_of_days('seven years ago', '2022-06-04'))
print(time_interval_with_number_of_days('yesterday', '2022-06-04'))
print(time_interval_with_number_of_days('1 Dec', '2022-06-04'))
print(time_interval_with_number_of_days('1 Dec 2020', '2022-06-04'))
print(time_interval_with_number_of_days('2 pm', '2022-06-08'))


(datetime.datetime(2021, 6, 7, 0, 0), datetime.datetime(2021, 6, 7, 23, 59))
(datetime.datetime(2015, 6, 9, 0, 0), datetime.datetime(2015, 6, 9, 23, 59))
(datetime.datetime(2022, 6, 3, 0, 0), datetime.datetime(2022, 6, 3, 23, 59))
(datetime.datetime(2021, 12, 1, 0, 0), datetime.datetime(2021, 12, 1, 23, 59))
(datetime.datetime(2020, 12, 1, 0, 0), datetime.datetime(2020, 12, 1, 23, 59))
(None, None)


In [97]:
# last week last month
# first week of last month
# third week of last month
# first week this month
# second week this month
# “The first week of the month” means the first complete week.

def week_month(phrase, date_string):
    
    last_week = False
    last_month = False
    first = False
    second = False
    third = False
    month = None
    date_beginning = None
    date_end = None
    
    split_phrase = phrase.split(' ')
    for i, word in enumerate(split_phrase):
        if word == 'week' and i > 0: 
            if split_phrase[i-1] == 'last':
                last_week = True
            elif split_phrase[i-1] == 'first':
                first = True
            elif split_phrase[i-1] == 'second':
                second = True 
            elif split_phrase[i-1] == 'third':
                third = True 
        if word == 'week' and ('last' in split_phrase[i:] or 'past' in split_phrase[i:]):
            last_month = True
        if word in months.keys():
            month = word
            
    if 'month' in phrase:         
        month = month_transformation(phrase)
            
    if last_month:
        date_1, date_2 = month_parsing('last ' + month, date_string)
    else:
        date_1, date_2 = month_parsing('this ' + month, date_string)
    
    date_beginning, date_end = date_1, date_2
    
    if date_1.weekday() <= 3: 
        partial_date_2 = date_1 + relativedelta(days = 6 - date_1.weekday())
    else:
        partial_date_2 = date_1 + relativedelta(days = 6 - date_1.weekday() + 7)
    if first:
        date_2 = partial_date_2
        
    if second:
        date_1 = partial_date_2 +  relativedelta(days = 1)
        date_2 = partial_date_2 + relativedelta(days = 7)
        
    if third:
        date_1 = partial_date_2 + relativedelta(days = 8)
        date_2 = partial_date_2 + relativedelta(days = 14)
        
    if last_week:

        if date_2.weekday() <= 3:
            date_1 = date_2 - relativedelta(days = (date_2.weekday() + 7))
        else:
            date_1 = date_2 - relativedelta(days = date_2.weekday())
            
    if date_1.month < months[month]:
        date_1 = date_beginning
    if date_2.month > months[month]:
        date_2 = date_end

    return date_1, date_2

print(week_month('last week last month', '2022-06-06'))
print(week_month('first week last month', '2022-06-06'))

print(week_month('second week last April', '2022-06-06'))
print(week_month('second week last month', '2022-06-06')) 
print(week_month('third week last March', '2022-06-06'))
print(week_month('last week this month', '2022-06-06'))

(datetime.date(2022, 5, 23), datetime.date(2022, 5, 31))
(datetime.date(2022, 5, 1), datetime.date(2022, 5, 8))
(datetime.date(2022, 4, 11), datetime.date(2022, 4, 17))
(datetime.date(2022, 5, 9), datetime.date(2022, 5, 15))
(datetime.date(2022, 3, 14), datetime.date(2022, 3, 20))
(datetime.date(2022, 6, 20), datetime.date(2022, 6, 30))


In [99]:
# last weekend of last month
# first weekend of this month
d = '2022-06-04'
def weekend_month(phrase, date_string):
    
    last_weekend = False
    last_month = False
    first_weekend = False
    second_weekend = False
    third_weekend = False
    month = None
    
    date_beginning = None
    date_end = None
    
    split_phrase = phrase.split(' ')
    for i, word in enumerate(split_phrase):
        if word == 'weekend' and i > 0: 
            if split_phrase[i-1] == 'last':
                last_weekend = True
            elif split_phrase[i-1] == 'first':
                first_weekend = True
            elif split_phrase[i-1] == 'second':
                second_weekend = True 
            elif split_phrase[i-1] == 'third':
                third_weekend = True 
        if word == 'weekend' and ('last' in split_phrase[i:] or 'past' in split_phrase[i:]):
            last_month = True
        if word in months.keys():
            month = word
    if 'month' in phrase:         
        month = month_transformation(phrase)
    if last_month:
        date_1, date_2 = month_parsing('last ' + month, date_string)
    else:
        date_1, date_2 = month_parsing('this ' + month, date_string)

    date_beginning, date_end = date_1, date_2
    current_weekday = date_beginning.weekday()
    last_day_weekday = date_end.weekday()

    if current_weekday < 5:
        first_weekend_date = date_1 + relativedelta(days = 5 - current_weekday)
    else:
        first_weekend_date = date_1 + relativedelta(days = 7 + 5 - current_weekday)
    
    if first_weekend:
        date_1 = first_weekend_date 
        date_2 = date_1 + relativedelta(days = 1, hours = 23, minutes = 59)
    if second_weekend:
        date_1 = first_weekend_date + relativedelta(days = 7)
        date_2 = date_1 + relativedelta(days = 1, hours = 23, minutes = 59)
    if third_weekend:
        date_1 = first_weekend_date + relativedelta(days = 14)
        date_2 = date_1 + relativedelta(days = 1, hours = 23, minutes = 59)
    if last_weekend:
        if last_day_weekday >= 5:
            date_1 = date_2 - relativedelta(days = abs(5 - last_day_weekday))
            date_2 = date_1 + relativedelta(days = 1, hours = 23, minutes = 59)
        else:
            date_1 = date_2 - relativedelta(days = last_day_weekday + 2)
            date_2 = date_1 + relativedelta(days = 1, hours = 23, minutes = 59)
    
    return date_1, date_2
            

print(weekend_month('second weekend this month', d))        
print(weekend_month('last weekend this month', d)) 
print(weekend_month('this weekend this month', d)) 
print(weekend_month('last weekend last month', d))
print(weekend_month('last weekend last May', d))
print(weekend_month('last weekend this April', d))
print(weekend_month('last weekend this April', d))

(datetime.date(2022, 6, 11), datetime.datetime(2022, 6, 12, 23, 59))
(datetime.date(2022, 6, 25), datetime.datetime(2022, 6, 26, 23, 59))
(datetime.date(2022, 6, 1), datetime.date(2022, 6, 30))
(datetime.date(2022, 5, 28), datetime.datetime(2022, 5, 29, 23, 59))
(datetime.date(2022, 5, 28), datetime.datetime(2022, 5, 29, 23, 59))
(datetime.date(2022, 4, 30), datetime.datetime(2022, 5, 1, 23, 59))
(datetime.date(2022, 4, 30), datetime.datetime(2022, 5, 1, 23, 59))


In [530]:
def specfied_time_intergation(date_start, date_end, time_start, time_end, date_string):

    if date_start and date_end:
        date_start_updated = dt.date(date_start.year, date_start.month, date_start.day) + relativedelta(hours=time_start.hour, minutes=time_start.minute)
        date_end_updated = dt.date(date_end.year, date_end.month, date_end.day) + relativedelta(hours=time_end.hour, minutes=time_end.minute)
    else:
        date_start = parse(date_string)
        date_start_updated = dt.date(date_start.year, date_start.month, date_start.day) + relativedelta(hours=time_start.hour, minutes=time_start.minute)
        date_end_updated = dt.date(date_start.year, date_start.month, date_start.day) + relativedelta(hours=time_end.hour, minutes=time_end.minute)
    return date_start_updated, date_end_updated
           

In [531]:
def exact_date_checker(phrase):
    date_definer = ['ago', 'from now', 'before', 'yesterday', 'today', 'tonight']
    if any(word in phrase for word in date_definer):
        return True
    try:
        return bool(parser.parse(phrase))
    except:
        return None
    
print(exact_date_checker('1 Dec'))
print(exact_date_checker('yesterday'))
print(exact_date_checker('4 pm 1 Dec'))
print(exact_date_checker('4 pm'))

True
True
True
True


In [529]:
vague_language = ['several', 'some', 'couple']
# specified dates, years and integration with time
def time_date_normalization(phrase, date_string):
    
    publication_date = parse(date_string)
    weekday = False
    week_in_phrase = False
    month_name = False
    month_in_phrase = False
    day_time = False
    exact_date = False
    weekend_in_phrase = False
    between_in_phrase = False
    exact_time = False
    year = False
    season = False
    number_of_years = 0
    vague_phrase = False
    
    date_start, date_end = None, None
    time_start, time_end = None, None
    
    split_phrase = phrase.split(' ')
    if re.search(regex_time, phrase):
        exact_time = True
        
    try:
        if exact_date_checker(phrase):
            exact_date = True
    except:
        pass
    
    for word in split_phrase:
        if word == 'weekend':
            weekend_in_phrase = True 
        elif word == 'week':
            week_in_phrase = True
        elif word == 'month':
            month_in_phrase = True
        elif word in times_of_day.keys():
            day_time = True
        elif word in months.keys():
            month_name = True
        elif word in week_days.keys():
            weekday = True 
        elif word in seasons.keys():
            season = True
        elif 'between' in word:
            between_in_phrase = True
        elif word in vague_language and 'ago' in phrase:
             vague_phrase = True
        try:
            possible_year_or_date = int(word)
            if len(word) == 4:
                year = True
                number_of_years += 1
        except:
            pass
#     if days_count(phrase):
#         exact_date = True
#     print(weekday, week_in_phrase, month_name, month_in_phrase, day_time, exact_date, weekend_in_phrase, between_in_phrase, exact_time, year, season)
    
    if between_in_phrase and (month_name or month_in_phrase or year or season or day_time):
        date_start, date_end = between_phrase(phrase, date_string)
        
    elif weekend and (month_in_phrase or month_name):
        date_start, date_end = weekend_month(phrase, date_string)
        
    elif week and (month_in_phrase or month_name):
        date_start, date_end = week_month(phrase, date_string)
        
    elif weekday or day_time:
        date_start, date_end = days_time_triplet(phrase, date_string)
        
    elif month_in_phrase or month_name and not vague_phrase:
        date_start, date_end = month_parsing(phrase, date_string)
        
    elif weekend_in_phrase and not (weekday and week_in_phrase and month_name and month_in_phrase and day_time and exact_date and between_phrase and exact_time and year and season):
        date_start, date_end = weekend(phrase, date_string)
        
    elif week_in_phrase and not (weekday and weekend_in_phrase and month_name and month_in_phrase and day_time and exact_date and between_phrase and exact_time and year and season):
        date_start, date_end = week(phrase, date_string)
    
    elif season and not (weekday and weekend_in_phrase and month_name and month and day_time and exact_date and between_phrase and exact_time and year and week):
        date_start, date_end = seasons_interval(phrase, date_string)
        
    elif exact_date and not (vague_phrase or weekday and weekend_in_phrase and month_name and month and day_time and exact_date and between_phrase and season and year and week):
        date_start, date_end = time_interval_with_number_of_days(phrase, date_string)
        
    elif year and number_of_years == 1:
        date_start, date_end = specific_year(phrase)
       
    elif vague_phrase:
        date_start, date_end = years_interval(phrase, date_string)

         
    if exact_time:
        time_start, time_end = time(phrase)
    
    if time_start and time_end:
        date_start_updated, date_end_updated  = specfied_time_intergation(date_start, date_end, time_start, time_end, date_string)
        return date_start_updated, date_end_updated
    else: 
        date_start = datetime.combine(date_start, datetime.min.time())
        date_end = date_end + relativedelta(hours=23, minutes=59)
    if date_start > parse(date_string):
        return None
    if date_end > parse(date_string) and date_start <= parse(date_string):
        return date_start, parse(date_string) + relativedelta(hours=23, minutes=59)

    return date_start, date_end

print('Between phrase: ')
print(time_date_normalization('between last evening and last night', '2022-06-06'))
print('Weekend + month: ')
print(time_date_normalization('last weekend this April', '2022-06-06'))
print(time_date_normalization('last weekend last April', '2022-06-06'))
print('Week + month: ')
print(time_date_normalization('second week last April', '2022-06-06'))
print('Week day + day time: ')
print(time_date_normalization('late Saturday evening', '2022-06-07'))
print(time_date_normalization('last October', '2022-06-07'))
print(time_date_normalization('last weekend', '2022-06-07'))
print(time_date_normalization('last week', '2022-06-07'))
print(time_date_normalization('this week', '2022-06-07'))
print('Seasons: ')
print(time_date_normalization("this winter", '2022-06-06'))
print(time_date_normalization("this summer", '2022-06-06'))
print('Weekday + time / precise date: ')
print(time_date_normalization("Monday", '2022-06-08'))
print(time_date_normalization("4 pm Monday", '2022-06-08'))
print(time_date_normalization("4 pm last Monday", '2022-06-08'))
print(time_date_normalization("Dec 15", '2022-06-08'))
print(time_date_normalization("4 pm Dec 15", '2022-06-08'))
print(time_date_normalization("4 pm", '2022-06-08'))
print(time_date_normalization("yesterday", '2022-06-08'))
print(time_date_normalization("today", '2022-06-08'))
print(time_date_normalization("weekend", '2022-06-08'))
print('Years: ')
print(time_date_normalization("in 2010", '2022-06-08'))
print(time_date_normalization("in 2022", '2022-06-08'))
print('Vague phrases: ')
print(time_date_normalization("several months ago", '2022-06-08'))
print(time_date_normalization("some days ago", '2022-06-08'))
print(time_date_normalization("a couple of years ago", '2022-06-08'))

Between phrase: 
(datetime.datetime(2022, 6, 5, 0, 0), datetime.datetime(2022, 6, 6, 23, 59))
Weekend + month: 
(datetime.datetime(2022, 4, 30, 0, 0), datetime.datetime(2022, 5, 2, 23, 58))
(datetime.datetime(2022, 4, 30, 0, 0), datetime.datetime(2022, 5, 2, 23, 58))
Week + month: 
(datetime.datetime(2022, 4, 1, 0, 0), datetime.datetime(2022, 4, 30, 23, 59))
Week day + day time: 
(datetime.datetime(2022, 6, 4, 0, 0), datetime.datetime(2022, 6, 5, 21, 59))
(datetime.datetime(2021, 10, 1, 0, 0), datetime.datetime(2021, 10, 31, 23, 59))
(datetime.datetime(2022, 6, 4, 0, 0), datetime.datetime(2022, 6, 6, 23, 58))
(datetime.datetime(2022, 5, 30, 0, 0), datetime.datetime(2022, 6, 6, 23, 58))
(datetime.datetime(2022, 6, 6, 0, 0), datetime.datetime(2022, 6, 7, 23, 59))
Seasons: 
(datetime.datetime(2021, 12, 1, 0, 0), datetime.datetime(2022, 3, 1, 23, 59))
(datetime.datetime(2022, 6, 1, 0, 0), datetime.datetime(2022, 6, 6, 23, 59))
Weekday + time / precise date: 
(datetime.datetime(2022, 6, 6, 